In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import statsmodels.api as sm 

import warnings

## Import Geospatial data

In [2]:
# Load the shapefile of LSOA of London
lsoa_gdf = gpd.read_file('../GIS_file/LSOA_2011_London_gen_MHW.shp')

# load Santander bike trip destination data for cluster0 and cluster1
endpoints0 = gpd.read_file('../GIS_file/end_points0.geojson')
endpoints1 = gpd.read_file('../GIS_file/end_points1.geojson')
endpointsALL = gpd.read_file('../GIS_file/end_pointsALL.geojson')

# imput polygon data
retail = gpd.read_file('../GIS_file/POI/landuse_retail.shp')
leisurepark = gpd.read_file('../GIS_file/POI/leisure_park.shp')
commercial = gpd.read_file('../GIS_file/POI/landuse_commercial1.shp')
residential = gpd.read_file('../GIS_file/POI/landuse_resident.shp')

# imput point data
office = gpd.read_file('../GIS_file/POI/office.shp')
school = gpd.read_file('../GIS_file/POI/school.shp')
tourism = gpd.read_file('../GIS_file/POI/tourism4.shp')
bus = gpd.read_file('../GIS_file/POI/bus.shp')
railway = gpd.read_file('../GIS_file/POI/railway_station.shp')
subway = gpd.read_file('../GIS_file/POI/subway_entrance1.shp')
PnR_spaces = gpd.read_file('../3.access/cycleparking_point.geojson')

# imput  road network data
roadnet = gpd.read_file('../GIS_file/road_network/roadnet_santander.shp') # polyline
roadjuctions = gpd.read_file('../GIS_file/road_network/roadnet_santander_ND_Junctions.shp') # point data

/opt/conda/lib/python3.10/site-packages/fiona/collection.py:208: FeatureWarning: Empty field name at index 0
  self._schema = self.session.get_schema()
/opt/conda/lib/python3.10/site-packages/fiona/collection.py:208: FeatureWarning: Empty field name at index 1
  self._schema = self.session.get_schema()
/opt/conda/lib/python3.10/site-packages/fiona/collection.py:208: FeatureWarning: Empty field name at index 3
  self._schema = self.session.get_schema()
/opt/conda/lib/python3.10/site-packages/fiona/collection.py:208: FeatureWarning: Empty field name at index 4
  self._schema = self.session.get_schema()
/opt/conda/lib/python3.10/site-packages/fiona/collection.py:208: FeatureWarning: Empty field name at index 5
  self._schema = self.session.get_schema()
/opt/conda/lib/python3.10/site-packages/fiona/collection.py:208: FeatureWarning: Empty field name at index 6
  self._schema = self.session.get_schema()
/opt/conda/lib/python3.10/site-packages/fiona/collection.py:208: FeatureWarning: Empty f

In [3]:
lsoa_gdf.head(2)

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry
0,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,1465,1465,0,112.9,876,1.7,"POLYGON ((532105.092 182011.230, 532162.491 18..."
1,E01000002,City of London 001B,E02000001,City of London 001,E09000001,City of London,E12000007,London,1436,1436,0,62.9,830,1.7,"POLYGON ((532746.813 181786.891, 532671.688 18..."


In [4]:
print(lsoa_gdf.crs)
print(endpoints0.crs)
print(endpoints1.crs)
print(retail.crs)
print(leisurepark.crs)
print(commercial.crs)
print(residential.crs)
print(school.crs)
print(office.crs)

epsg:27700
epsg:27700
epsg:27700
epsg:27700
epsg:27700
epsg:27700
epsg:27700
epsg:27700
epsg:27700


### 统计LSOA内各要素的总面积

In [5]:
commercial['area'] = commercial.geometry.area

retail['area'] = retail.geometry.area

leisurepark['area'] = leisurepark.geometry.area

residential['area'] = residential.geometry.area

school['area'] = school.geometry.area
school.head(2)

,full_id,osm_id,osm_type,school,wheelchair,contact_we,addr_unit,addr_stree,addr_postc,addr_house,...,website,motor_vehi,horse,access,name,amenity,entrance,barrier,geometry,area
0,n27281524,27281524,node,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,gate,POINT (534533.628 197241.246),0.0
1,n129944384,129944384,node,None,None,None,None,None,None,None,...,None,None,None,None,None,None,main,gate,POINT (532504.496 170003.453),0.0


In [6]:
lsoa_gdf['SHAPE_area'] = lsoa_gdf.geometry.area
#lsoa_gdf['PopDensity'] = lsoa_gdf['USUALRES'].astype(int) / lsoa_gdf['SHAPE_area'].astype(float)

In [7]:
lsoa_gdf.head(1)

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry,SHAPE_area
0,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,1465,1465,0,112.9,876,1.7,"POLYGON ((532105.092 182011.230, 532162.491 18...",133320.768859


In [8]:
lsoa_gdf = lsoa_gdf[['LSOA11CD', 'USUALRES', 'SHAPE_area', 'geometry']]
lsoa_gdf.head(1)

,LSOA11CD,USUALRES,SHAPE_area,geometry
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18..."


In [9]:
joined_gdf1 = gpd.sjoin(commercial, lsoa_gdf, how="inner", predicate="intersects")
joined_gdf1.head()

# 求和每个LSOA单元内的commercial polygon面积
area_sum = joined_gdf1.groupby('LSOA11CD')['area'].sum().reset_index()

# 将求和结果合并回lsoa_gdf
lsoa_gdf  = lsoa_gdf.merge(area_sum, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的commercial polygons，它们的commercial_area_sum可能是NaN，可以将这些值设为0
lsoa_gdf['area'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'area': 'commercial_area'})

# 查看结果
lsoa_gdf.head(1)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979


In [10]:
joined_gdf2 = gpd.sjoin(residential, lsoa_gdf, how="inner", predicate="intersects")
joined_gdf2.head()

# 求和每个LSOA单元内的commercial polygon面积
area_sum = joined_gdf2.groupby('LSOA11CD')['area'].sum().reset_index()

# 将求和结果合并回lsoa_gdf
lsoa_gdf  = lsoa_gdf.merge(area_sum, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的commercial polygons，它们的commercial_area_sum可能是NaN，可以将这些值设为0
lsoa_gdf['area'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'area': 'residential_area'})

# 查看结果
lsoa_gdf.head(1)

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0


In [11]:
joined_gdf3 = gpd.sjoin(leisurepark, lsoa_gdf, how="inner", predicate="intersects")
joined_gdf3.head()

# 求和每个LSOA单元内的commercial polygon面积
area_sum = joined_gdf3.groupby('LSOA11CD')['area'].sum().reset_index()

# 将求和结果合并回lsoa_gdf
lsoa_gdf  = lsoa_gdf.merge(area_sum, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的commercial polygons，它们的commercial_area_sum可能是NaN，可以将这些值设为0
lsoa_gdf['area'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'area': 'leisurepark_area'})

# 查看结果
lsoa_gdf.head(2)

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471


In [12]:
joined_gdf4 = gpd.sjoin(retail, lsoa_gdf, how="inner", predicate="intersects")
joined_gdf4.head()

# 求和每个LSOA单元内的commercial polygon面积
area_sum = joined_gdf4.groupby('LSOA11CD')['area'].sum().reset_index()

# 将求和结果合并回lsoa_gdf
lsoa_gdf  = lsoa_gdf.merge(area_sum, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的commercial polygons，它们的commercial_area_sum可能是NaN，可以将这些值设为0
lsoa_gdf['area'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'area': 'retail_area'})

# 查看结果
lsoa_gdf.head(2)

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472


### 计算LSOA中点的数量

In [13]:
joined_point1 = gpd.sjoin(school, lsoa_gdf, how="inner", predicate="intersects")
joined_point1.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point1.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'school_point'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0


In [14]:
joined_point2 = gpd.sjoin(office, lsoa_gdf, how="inner", predicate="intersects")
joined_point2.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point2.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'office_point'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0


In [15]:
joined_point3 = gpd.sjoin(bus, lsoa_gdf, how="inner", predicate="intersects")
joined_point3.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point3.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'bus_point'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0


In [16]:
joined_point4 = gpd.sjoin(railway, lsoa_gdf, how="inner", predicate="intersects")
joined_point4.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point4.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'railway_point'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0


In [17]:
joined_point5 = gpd.sjoin(subway, lsoa_gdf, how="inner", predicate="intersects")
joined_point5.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point5.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'subway_point'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0


In [18]:
joined_point6 = gpd.sjoin(tourism, lsoa_gdf, how="inner", predicate="intersects")
joined_point6.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point6.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'tourism_point'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0


In [19]:
joined_point7 = gpd.sjoin(endpointsALL, lsoa_gdf, how="inner", predicate="intersects")
joined_point7.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point7.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'endpointsALL'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0


In [43]:
joined_point77 = gpd.sjoin(endpoints0, lsoa_gdf, how="inner", predicate="intersects")
joined_point77.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point77.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'endpoints0'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,...,NoCar,OfficeWorkingResidents,MetroCommute,BusCommute,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST,endpoints0
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,...,0.659880,0.150943,0.156250,0.053571,0.080357,0.107143,0.000000,0.000000,0.000,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,...,0.703524,0.148336,0.146341,0.073171,0.092683,0.121951,68.699428,202.332489,222.974,1.0


In [44]:
joined_point777 = gpd.sjoin(endpoints1, lsoa_gdf, how="inner", predicate="intersects")
joined_point777.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point777.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'endpoints1'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,...,OfficeWorkingResidents,MetroCommute,BusCommute,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST,endpoints0,endpoints1
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,...,0.150943,0.156250,0.053571,0.080357,0.107143,0.000000,0.000000,0.000,0.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,...,0.148336,0.146341,0.073171,0.092683,0.121951,68.699428,202.332489,222.974,1.0,2.0


In [20]:
joined_point8 = gpd.sjoin(PnR_spaces, lsoa_gdf, how="inner", predicate="intersects")
joined_point8.head()

# 计算每个LSOA单元内的点数量
point_count = joined_point7.groupby('LSOA11CD').size().reset_index(name='point_count')

# 将计数结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(point_count, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的点，它们的point_count可能是NaN，可以将这些值设为0
lsoa_gdf['point_count'].fillna(0, inplace=True)

lsoa_gdf = lsoa_gdf.rename(columns={'point_count': 'PnR_spaces'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL,PnR_spaces
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0,2.0


In [21]:
lsoa_gdf.fillna(0, inplace=True)
lsoa_gdf.head()

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL,PnR_spaces
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.000000,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.000000,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0,2.0
2,E01000003,1346,57302.966528,"POLYGON ((532135.145 182198.119, 532158.250 18...",0.000000,23724.679983,5038.305758,2014.188703,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0
3,E01000005,985,190738.760508,"POLYGON ((533807.946 180767.770, 533649.063 18...",105496.231326,16518.797082,2371.863605,5026.378876,0.0,7.0,15.0,2.0,3.0,17.0,1.0,1.0
4,E01000006,1703,144195.846859,"POLYGON ((545122.049 184314.931, 545271.917 18...",0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 计算LSOA中停放点的个数×一个月内每个点的骑行量

In [22]:
endpoints0.head(2)

,EndStation Id,inbound_counts,id,name,lat,lon,capacity,geometry
0,39,767,39,"Shoreditch High Street, Shoreditch",51.526378,-0.078131,3,POINT (533420.107 182579.821)
1,53,74,53,"Grafton Street, Mayfair",51.509992,-0.143495,2,POINT (528932.257 180640.623)


In [23]:
# 执行空间连接
joined_count1 = gpd.sjoin(endpoints0, lsoa_gdf, how="inner", predicate="intersects")

# 计算每个LSOA单元内的inbound_count之和
inbound_sum = joined_count1.groupby('LSOA11CD')['inbound_counts'].sum().reset_index(name='inbound_count_sum')

# 将求和结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(inbound_sum, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的endpoints0，它们的inbound_count_sum可能是NaN，可以将这些值设为0
lsoa_gdf['inbound_count_sum'].fillna(0, inplace=True)

# 重命名列以反映它表示的是inbound_count的总和
lsoa_gdf = lsoa_gdf.rename(columns={'inbound_count_sum': 'endpoints0_count'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL,PnR_spaces,endpoints0_count
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0,0.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0,2.0,80.0


In [24]:
# 执行空间连接
joined_count2= gpd.sjoin(endpoints1, lsoa_gdf, how="inner", predicate="intersects")

# 计算每个LSOA单元内的inbound_count之和
inbound_sum = joined_count2.groupby('LSOA11CD')['inbound_counts'].sum().reset_index(name='inbound_count_sum')

# 将求和结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(inbound_sum, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的endpoints0，它们的inbound_count_sum可能是NaN，可以将这些值设为0
lsoa_gdf['inbound_count_sum'].fillna(0, inplace=True)

# 重命名列以反映它表示的是inbound_count的总和
lsoa_gdf = lsoa_gdf.rename(columns={'inbound_count_sum': 'endpoints1_count'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL,PnR_spaces,endpoints0_count,endpoints1_count
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0,2.0,80.0,271.0


In [25]:
# 执行空间连接
joined_count3= gpd.sjoin(endpointsALL, lsoa_gdf, how="inner", predicate="intersects")

# 计算每个LSOA单元内的inbound_count之和
inbound_sum = joined_count3.groupby('LSOA11CD')['inbound_counts'].sum().reset_index(name='inbound_count_sum')

# 将求和结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(inbound_sum, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的endpoints0，它们的inbound_count_sum可能是NaN，可以将这些值设为0
lsoa_gdf['inbound_count_sum'].fillna(0, inplace=True)

# 重命名列以反映它表示的是inbound_count的总和
lsoa_gdf = lsoa_gdf.rename(columns={'inbound_count_sum': 'endpointsALL_count'})

# 查看结果
lsoa_gdf.head(2)


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL,PnR_spaces,endpoints0_count,endpoints1_count,endpointsALL_count
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0,2.0,80.0,271.0,351.0


### 计算LSOA中polyline的长度

In [26]:
roadnet = gpd.read_file('../GIS_file/road_network/roadnet_santander.shp') # polyline

In [27]:
roadnet.head(2)

,fid_1,u,v,key,osmid,name,highway,maxspeed,access,oneway,...,ref,service,bridge,junction,tunnel,width,est_width,landuse,area,geometry
0,1.0,78112.0,25508583.0,0.0,129375498,Outer Circle,unclassified,20 mph,permissive,0,...,None,None,None,None,None,None,None,None,None,"LINESTRING (528724.368 182505.622, 528723.253 ..."
1,2.0,78112.0,25508584.0,0.0,129375498,Outer Circle,unclassified,20 mph,permissive,0,...,None,None,None,None,None,None,None,None,None,"LINESTRING (528720.628 182588.803, 528723.253 ..."


In [28]:
roadnet['length'] = roadnet.geometry.length
#roadnet.head(2)

# spatial join LSOA和residential数据
joined_line = gpd.sjoin(roadnet, lsoa_gdf, how="inner", predicate="intersects")
joined_line.head()
# 求和每个LSOA单元内的polyline长度
# 假设residential GeoDataFrame中已经有了一个名为'length'的列，它存储了每个polyline的长度
length_sum = joined_line.groupby('LSOA11CD')['length'].sum().reset_index()
# 将求和结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(length_sum, on='LSOA11CD', how='left')
# 如果某些LSOA单元没有匹配的polyline，它们的length_sum可能是NaN，可以将这些值设为0
lsoa_gdf['length'].fillna(0, inplace=True)
# 重命名列
lsoa_gdf = lsoa_gdf.rename(columns={'length': 'road_length'})
# 查看结果
lsoa_gdf.head()

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL,PnR_spaces,endpoints0_count,endpoints1_count,endpointsALL_count,road_length
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.000000,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.000000,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0,2.0,80.0,271.0,351.0,88.828273
2,E01000003,1346,57302.966528,"POLYGON ((532135.145 182198.119, 532158.250 18...",0.000000,23724.679983,5038.305758,2014.188703,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,102.563588
3,E01000005,985,190738.760508,"POLYGON ((533807.946 180767.770, 533649.063 18...",105496.231326,16518.797082,2371.863605,5026.378876,0.0,7.0,15.0,2.0,3.0,17.0,1.0,1.0,65.0,667.0,732.0,1418.000350
4,E01000006,1703,144195.846859,"POLYGON ((545122.049 184314.931, 545271.917 18...",0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [29]:
lsoa_gdf.head()

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,subway_point,tourism_point,endpointsALL,PnR_spaces,endpoints0_count,endpoints1_count,endpointsALL_count,road_length
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.000000,4037.153500,0.000000,0.0,2.0,7.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.000000,3538.595471,13099.470472,0.0,9.0,6.0,1.0,1.0,14.0,2.0,2.0,80.0,271.0,351.0,88.828273
2,E01000003,1346,57302.966528,"POLYGON ((532135.145 182198.119, 532158.250 18...",0.000000,23724.679983,5038.305758,2014.188703,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,102.563588
3,E01000005,985,190738.760508,"POLYGON ((533807.946 180767.770, 533649.063 18...",105496.231326,16518.797082,2371.863605,5026.378876,0.0,7.0,15.0,2.0,3.0,17.0,1.0,1.0,65.0,667.0,732.0,1418.000350
4,E01000006,1703,144195.846859,"POLYGON ((545122.049 184314.931, 545271.917 18...",0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


## 加入各LSOA2021Cencus Socioeconomic and demographic data

In [30]:
socioeconomic = pd.read_csv("../LSOA_alldata4.csv")
#lsoa_gdf = gpd.read_file('../GIS_file/LSOA_2011_London_gen_MHW.shp')
socioeconomic.head(2)

,local_authority_code,local_authority_name,LSOA_ID,AllusualResidents,JobDensity,StudentRatio,Deprivation,NoCar,OfficeWorkingResidents,MetroCommute,BusCommute,CarorVanCommute,CycleCommute
0,E09000001,City of London,E01000001,1484,0.910767,0.061947,0.003580,0.659880,0.150943,0.156250,0.053571,0.080357,0.107143
1,E09000001,City of London,E01000002,1382,0.900154,0.067588,0.001214,0.703524,0.148336,0.146341,0.073171,0.092683,0.121951


In [31]:
# Merge the socioeconomic data with the LSOA shapefile based on matching LSOA codes
lsoa_gdf = pd.merge(lsoa_gdf, socioeconomic, left_on='LSOA11CD', right_on='LSOA_ID', how='left')

# Print the first few rows of the merged data to verify
lsoa_gdf.head(2)

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,...,AllusualResidents,JobDensity,StudentRatio,Deprivation,NoCar,OfficeWorkingResidents,MetroCommute,BusCommute,CarorVanCommute,CycleCommute
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,...,1484.0,0.910767,0.061947,0.003580,0.659880,0.150943,0.156250,0.053571,0.080357,0.107143
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,...,1382.0,0.900154,0.067588,0.001214,0.703524,0.148336,0.146341,0.073171,0.092683,0.121951


In [32]:
lsoa_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4835 entries, 0 to 4834
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   LSOA11CD                4835 non-null   object  
 1   USUALRES                4835 non-null   int64   
 2   SHAPE_area              4835 non-null   float64 
 3   geometry                4835 non-null   geometry
 4   commercial_area         4835 non-null   float64 
 5   residential_area        4835 non-null   float64 
 6   leisurepark_area        4835 non-null   float64 
 7   retail_area             4835 non-null   float64 
 8   school_point            4835 non-null   float64 
 9   office_point            4835 non-null   float64 
 10  bus_point               4835 non-null   float64 
 11  railway_point           4835 non-null   float64 
 12  subway_point            4835 non-null   float64 
 13  tourism_point           4835 non-null   float64 
 14  endpointsALL    

In [33]:
# 删除 local_authority_name 和 local_authority_code 列
lsoa_gdf = lsoa_gdf.drop(columns=['local_authority_name', 'local_authority_code'])
lsoa_gdf.head(1)

,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,...,AllusualResidents,JobDensity,StudentRatio,Deprivation,NoCar,OfficeWorkingResidents,MetroCommute,BusCommute,CarorVanCommute,CycleCommute
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.1535,0.0,0.0,2.0,...,1484.0,0.910767,0.061947,0.00358,0.65988,0.150943,0.15625,0.053571,0.080357,0.107143


## 计算LSOA内各自行车停放站点到最近的地铁站距离的平均距离

In [34]:
distance_cal = pd.read_csv("../2_correlation/ALL_OSM_Rat4.csv")
distance_cal = distance_cal[['EndStationID','BUS_DIST', 'METRO_DIST', 'RAIL_DIST']]
distance_cal.head(2)

,EndStationID,BUS_DIST,METRO_DIST,RAIL_DIST
0,1,239.837470,495.102729,459.626131
1,2,67.422326,364.498885,387.149307


In [35]:
endpointsALL.head()

,EndStation Id,inbound_counts,id,name,lat,lon,capacity,geometry
0,581,711,581,"New Cavendish Street, Marylebone",51.519167,-0.147983,0,POINT (528595.027 181652.995)
1,361,721,361,"Waterloo Station 2, Waterloo",51.503920,-0.113426,0,POINT (531036.189 180018.691)
2,71,1308,71,"Newgate Street , St. Paul's",51.515419,-0.098851,0,POINT (532014.507 181323.574)
3,153,1723,153,"Bayswater Road, Hyde Park",51.511933,-0.174411,2,POINT (526781.583 180802.471)
4,752,1059,752,"London Street, Paddington",51.515117,-0.173713,1,POINT (526821.197 181157.754)


In [36]:
avg_distance = pd.merge(distance_cal, endpointsALL, left_on='EndStationID', right_on='EndStation Id', how='inner')
avg_distance.head(2)

,EndStationID,BUS_DIST,METRO_DIST,RAIL_DIST,EndStation Id,inbound_counts,id,name,lat,lon,capacity,geometry
0,1,239.837470,495.102729,459.626131,1,216,1,"River Street , Clerkenwell",51.529163,-0.109971,0,POINT (531203.517 182832.066)
1,3,89.621501,391.285362,410.602950,3,495,3,"Christopher Street, Liverpool Street",51.521284,-0.084606,0,POINT (532985.807 182001.572)


In [37]:
# 假设 avg_distance 是一个普通的 DataFrame，且有一个名为 geometry 的列存储了空间数据
avg_distance = gpd.GeoDataFrame(avg_distance, geometry='geometry')

# 执行空间连接
joined_avg = gpd.sjoin(avg_distance, lsoa_gdf, how="inner", op="intersects")

# 计算每个LSOA单元内的BUS_DIST, METRO_DIST, RAIL_DIST各自的平均值
distance_avg = joined_avg.groupby('LSOA11CD').agg({'BUS_DIST': 'mean', 'METRO_DIST': 'mean', 'RAIL_DIST': 'mean'}).reset_index()

# 将求和结果合并回lsoa_gdf
lsoa_gdf = lsoa_gdf.merge(distance_avg, on='LSOA11CD', how='left')

# 如果某些LSOA单元没有匹配的endpoints0，它们的inbound_count_sum可能是NaN，可以将这些值设为0
lsoa_gdf.fillna(0, inplace=True)

# 重命名列以反映它表示的是inbound_count的总和
lsoa_gdf = lsoa_gdf.rename(columns={'BUS_DIST': 'avg_BUS_DIST', 'METRO_DIST': 'avg_METRO_DIST', 'RAIL_DIST': 'avg_RAIL_DIST'})

# 查看结果
lsoa_gdf.head(2)

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,...,Deprivation,NoCar,OfficeWorkingResidents,MetroCommute,BusCommute,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST
0,E01000001,1465,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",166000.549979,0.0,4037.153500,0.000000,0.0,2.0,...,0.003580,0.659880,0.150943,0.156250,0.053571,0.080357,0.107143,0.000000,0.000000,0.000
1,E01000002,1436,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",186220.933756,0.0,3538.595471,13099.470472,0.0,9.0,...,0.001214,0.703524,0.148336,0.146341,0.073171,0.092683,0.121951,68.699428,202.332489,222.974


## export gdf to geojson

In [39]:
lsoa_gdf.describe()

,USUALRES,SHAPE_area,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,...,Deprivation,NoCar,OfficeWorkingResidents,MetroCommute,BusCommute,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST
count,4835.000000,4.835000e+03,4.835000e+03,4.835000e+03,4.835000e+03,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,...,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000
mean,1690.577249,3.254407e+05,2.060039e+03,1.213883e+05,1.449366e+05,13553.109641,0.007859,0.765460,0.737125,0.053981,...,0.044548,0.379063,0.274274,0.160275,0.139903,0.343835,0.054017,11.369193,32.430208,26.806716
std,263.566401,6.290090e+05,2.306790e+04,3.526299e+05,6.458418e+05,30877.657415,0.111132,3.203291,3.277248,0.305416,...,0.030004,0.197669,0.077300,0.102125,0.077017,0.174107,0.048404,59.299391,144.806209,117.031315
min,985.000000,1.690171e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1530.000000,1.335480e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020976,0.226679,0.239189,0.075023,0.085215,0.197925,0.016129,0.000000,0.000000,0.000000
50%,1654.000000,2.033552e+05,0.000000e+00,0.000000e+00,3.250805e+03,1417.218362,0.000000,0.000000,0.000000,0.000000,...,0.039264,0.361156,0.286370,0.155456,0.131507,0.330961,0.035120,0.000000,0.000000,0.000000
75%,1817.000000,3.190558e+05,0.000000e+00,1.011903e+05,6.195812e+04,11669.369780,0.000000,0.000000,0.000000,0.000000,...,0.064275,0.536725,0.325755,0.231823,0.186455,0.486987,0.084464,0.000000,0.000000,0.000000
max,4933.000000,1.580873e+07,1.363983e+06,3.860944e+06,1.609589e+07,399957.544983,3.000000,134.000000,130.000000,10.000000,...,0.200000,0.867347,0.466506,0.558528,0.465000,0.791753,0.290698,841.039976,1404.020820,1195.401746


In [45]:
lsoa_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4835 entries, 0 to 4834
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   LSOA11CD                4835 non-null   object  
 1   USUALRES                4835 non-null   int64   
 2   SHAPE_area              4835 non-null   float64 
 3   geometry                4835 non-null   geometry
 4   commercial_area         4835 non-null   float64 
 5   residential_area        4835 non-null   float64 
 6   leisurepark_area        4835 non-null   float64 
 7   retail_area             4835 non-null   float64 
 8   school_point            4835 non-null   float64 
 9   office_point            4835 non-null   float64 
 10  bus_point               4835 non-null   float64 
 11  railway_point           4835 non-null   float64 
 12  subway_point            4835 non-null   float64 
 13  tourism_point           4835 non-null   float64 
 14  endpointsALL    

In [41]:
lsoa_gdf = lsoa_gdf.drop(columns=['LSOA_ID', 'AllusualResidents'])

In [42]:
# 生成原始数据表
lsoa_gdf.to_file("regressionALL1.geojson", driver='GeoJSON')

In [46]:
#Average monthly number of DBS trips per station
lsoa_gdf['avg_tripsALL'] = lsoa_gdf['endpointsALL_count'] / lsoa_gdf['endpointsALL']
lsoa_gdf['avg_trips0'] = lsoa_gdf['endpoints0_count'] / lsoa_gdf['endpoints0']
lsoa_gdf['avg_trips1'] = lsoa_gdf['endpoints0_count'] / lsoa_gdf['endpoints1']

In [47]:
# 需要计算的列名列表
columns_to_calculate = ['USUALRES', 'road_length',
                        'commercial_area', 'residential_area', 'leisurepark_area','retail_area', 
                        'office_point','PnR_spaces','school_point','tourism_point',
                        'bus_point', 'subway_point','railway_point']

# 对于每个列，计算其值除以SHAPE_area，并重命名列
for column in columns_to_calculate:
    # 计算比例
    lsoa_gdf[column] = lsoa_gdf[column] / lsoa_gdf['SHAPE_area']     

lsoa_gdf.head()


,LSOA11CD,USUALRES,SHAPE_area,geometry,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,...,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST,endpoints0,endpoints1,avg_tripsALL,avg_trips0,avg_trips1
0,E01000001,0.010989,133320.768859,"POLYGON ((532105.092 182011.230, 532162.491 18...",1.245121,0.000000,0.030282,0.000000,0.0,0.000015,...,0.080357,0.107143,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN
1,E01000002,0.006349,226191.273063,"POLYGON ((532746.813 181786.891, 532671.688 18...",0.823290,0.000000,0.015644,0.057913,0.0,0.000040,...,0.092683,0.121951,68.699428,202.332489,222.974000,1.0,2.0,175.5,80.0,40.0
2,E01000003,0.023489,57302.966528,"POLYGON ((532135.145 182198.119, 532158.250 18...",0.000000,0.414022,0.087924,0.035150,0.0,0.000017,...,0.070130,0.161039,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN
3,E01000005,0.005164,190738.760508,"POLYGON ((533807.946 180767.770, 533649.063 18...",0.553093,0.086604,0.012435,0.026352,0.0,0.000037,...,0.108844,0.068027,95.222448,121.435602,103.816109,1.0,1.0,732.0,65.0,65.0
4,E01000006,0.011810,144195.846859,"POLYGON ((545122.049 184314.931, 545271.917 18...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.324247,0.008608,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN


In [48]:
# 将 avg_tripsALL、avg_trips0 和 avg_trips1 列中的空值替换为0
lsoa_gdf['avg_tripsALL'].fillna(0, inplace=True)
lsoa_gdf['avg_trips0'].fillna(0, inplace=True)
lsoa_gdf['avg_trips1'].fillna(0, inplace=True)
lsoa_gdf.describe()

,USUALRES,SHAPE_area,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,...,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST,endpoints0,endpoints1,avg_tripsALL,avg_trips0,avg_trips1
count,4835.000000,4.835000e+03,4835.000000,4835.000000,4835.000000,4835.000000,4.835000e+03,4835.000000,4835.000000,4.835000e+03,...,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000
mean,0.009599,3.254407e+05,0.010839,0.823706,0.396351,0.065378,4.317150e-08,0.000004,0.000004,3.107334e-07,...,0.343835,0.054017,11.369193,32.430208,26.806716,0.082523,0.094312,40.603290,9.465122,9.038432
std,0.006146,6.290090e+05,0.077240,2.222223,1.681537,0.152959,7.182363e-07,0.000013,0.000013,1.765168e-06,...,0.174107,0.048404,59.299391,144.806209,117.031315,0.387558,0.429804,182.387708,50.212228,47.543581
min,0.000116,1.690171e+04,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.005203,1.335480e+05,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.197925,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.008339,2.033552e+05,0.000000,0.000000,0.019767,0.005554,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.330961,0.035120,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.012814,3.190558e+05,0.000000,0.816222,0.223430,0.058694,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.486987,0.084464,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.074134,1.580873e+07,2.237729,31.225138,40.364013,2.184887,2.321242e-05,0.000400,0.000171,4.071919e-05,...,0.791753,0.290698,841.039976,1404.020820,1195.401746,12.000000,14.000000,2562.500000,1006.000000,1006.000000


In [49]:
lsoa_gdf = lsoa_gdf[lsoa_gdf['endpointsALL'] != 0]
lsoa_gdf.describe()

,USUALRES,SHAPE_area,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,...,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST,endpoints0,endpoints1,avg_tripsALL,avg_trips0,avg_trips1
count,327.000000,3.270000e+02,327.000000,327.000000,327.000000,327.000000,3.270000e+02,327.000000,327.000000,327.000000,...,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000
mean,0.013817,1.872836e+05,0.063248,1.398461,0.329287,0.076078,1.535298e-08,0.000018,0.000025,0.000002,...,0.136686,0.087155,168.104129,479.510877,396.362302,1.220183,1.394495,600.357514,139.950663,133.641648
std,0.006679,2.110989e+05,0.146695,2.012836,1.021978,0.134833,2.776300e-07,0.000026,0.000026,0.000003,...,0.065114,0.046671,160.354634,309.665808,236.991488,0.913677,0.959470,395.197884,138.088240,129.670094
min,0.000610,4.018281e+04,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,4.675317,5.294271,4.082762,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.009156,9.210312e+04,0.000000,0.504273,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.109129,0.059286,61.081707,237.353369,216.392991,1.000000,1.000000,327.083333,43.000000,42.000000
50%,0.013568,1.312771e+05,0.000000,0.880429,0.033603,0.021958,0.000000e+00,0.000009,0.000018,0.000000,...,0.146939,0.089947,110.145640,418.786979,374.047096,1.000000,1.000000,560.000000,103.000000,97.750000
75%,0.017548,1.876072e+05,0.055013,1.542509,0.157460,0.108405,0.000000e+00,0.000025,0.000037,0.000000,...,0.180197,0.116551,222.747904,677.181360,525.178024,1.000000,2.000000,813.250000,194.000000,190.500000
max,0.035836,1.922781e+06,1.376957,17.320731,8.743051,1.409971,5.020423e-06,0.000179,0.000171,0.000019,...,0.301954,0.221951,841.039976,1404.020820,1195.401746,12.000000,14.000000,2562.500000,1006.000000,1006.000000


In [50]:
# 需要计算的列名列表
columns_to_calculate1 = ['USUALRES', 'road_length',
                        'office_point','PnR_spaces','school_point','tourism_point',
                        'bus_point', 'subway_point','railway_point']

# 对于每个列，计算其值*1000000单位由m2转换成km2，并重命名列
for column in columns_to_calculate1:
    # 计算比例
    lsoa_gdf[column] = lsoa_gdf[column] *1000000
lsoa_gdf.describe()

,USUALRES,SHAPE_area,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,...,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST,endpoints0,endpoints1,avg_tripsALL,avg_trips0,avg_trips1
count,327.000000,3.270000e+02,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,...,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000
mean,13817.281239,1.872836e+05,0.063248,1.398461,0.329287,0.076078,0.015353,18.369942,24.724975,1.596677,...,0.136686,0.087155,168.104129,479.510877,396.362302,1.220183,1.394495,600.357514,139.950663,133.641648
std,6679.272637,2.110989e+05,0.146695,2.012836,1.021978,0.134833,0.277630,26.123272,26.480400,3.335446,...,0.065114,0.046671,160.354634,309.665808,236.991488,0.913677,0.959470,395.197884,138.088240,129.670094
min,610.053786,4.018281e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,4.675317,5.294271,4.082762,0.000000,1.000000,1.000000,0.000000,0.000000
25%,9155.831384,9.210312e+04,0.000000,0.504273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.109129,0.059286,61.081707,237.353369,216.392991,1.000000,1.000000,327.083333,43.000000,42.000000
50%,13567.716754,1.312771e+05,0.000000,0.880429,0.033603,0.021958,0.000000,9.054885,18.308117,0.000000,...,0.146939,0.089947,110.145640,418.786979,374.047096,1.000000,1.000000,560.000000,103.000000,97.750000
75%,17548.321482,1.876072e+05,0.055013,1.542509,0.157460,0.108405,0.000000,25.093680,37.243943,0.000000,...,0.180197,0.116551,222.747904,677.181360,525.178024,1.000000,2.000000,813.250000,194.000000,190.500000
max,35836.216283,1.922781e+06,1.376957,17.320731,8.743051,1.409971,5.020423,178.889632,171.489286,18.814849,...,0.301954,0.221951,841.039976,1404.020820,1195.401746,12.000000,14.000000,2562.500000,1006.000000,1006.000000


In [51]:
# 检查lsoa_gdf中的空值
null_columns = lsoa_gdf.columns[lsoa_gdf.isnull().any()].tolist()
# 对于有空值的列，将空值替换为0
for column in null_columns:
    lsoa_gdf[column].fillna(0, inplace=True)
print(f"被替换的空值: {null_columns}")

被替换的空值: []


In [52]:
# 检查重复的列名
duplicates = lsoa_gdf.columns[lsoa_gdf.columns.duplicated()]
print(duplicates)

# 导出未经单位转换的 lsoa_gdf 数据框
lsoa_gdf.to_file("regressionALL2.geojson", driver='GeoJSON')

Index([], dtype='object')


In [53]:
# 导出未经单位转换的 lsoa_gdf 数据框为 CSV 文件
lsoa_gdf.to_csv('regressionALL2.csv', index=False)

In [6]:
# Load trip data
# trips = pd.read_csv("ALL_OSM_Rat4.csv")
trips = gpd.read_file('regressionALL2.geojson')
trips.describe()

,USUALRES,SHAPE_area,commercial_area,residential_area,leisurepark_area,retail_area,school_point,office_point,bus_point,railway_point,...,CarorVanCommute,CycleCommute,avg_BUS_DIST,avg_METRO_DIST,avg_RAIL_DIST,endpoints0,endpoints1,avg_tripsALL,avg_trips0,avg_trips1
count,327.000000,3.270000e+02,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,...,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000
mean,13817.281239,1.872836e+05,0.063248,1.398461,0.329287,0.076078,0.015353,18.369942,24.724975,1.596677,...,0.136686,0.087155,168.104129,479.510877,396.362302,1.220183,1.394495,600.357514,139.950663,133.641648
std,6679.272637,2.110989e+05,0.146695,2.012836,1.021978,0.134833,0.277630,26.123272,26.480400,3.335446,...,0.065114,0.046671,160.354634,309.665808,236.991488,0.913677,0.959470,395.197884,138.088240,129.670094
min,610.053786,4.018281e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,4.675317,5.294271,4.082762,0.000000,1.000000,1.000000,0.000000,0.000000
25%,9155.831384,9.210312e+04,0.000000,0.504273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.109129,0.059286,61.081707,237.353369,216.392991,1.000000,1.000000,327.083333,43.000000,42.000000
50%,13567.716754,1.312771e+05,0.000000,0.880429,0.033603,0.021958,0.000000,9.054885,18.308117,0.000000,...,0.146939,0.089947,110.145640,418.786979,374.047096,1.000000,1.000000,560.000000,103.000000,97.750000
75%,17548.321482,1.876072e+05,0.055013,1.542509,0.157460,0.108405,0.000000,25.093680,37.243943,0.000000,...,0.180197,0.116551,222.747904,677.181360,525.178024,1.000000,2.000000,813.250000,194.000000,190.500000
max,35836.216283,1.922781e+06,1.376957,17.320731,8.743051,1.409971,5.020423,178.889632,171.489286,18.814849,...,0.301954,0.221951,841.039976,1404.020820,1195.401746,12.000000,14.000000,2562.500000,1006.000000,1006.000000


In [7]:
trips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   LSOA11CD                327 non-null    object  
 1   USUALRES                327 non-null    float64 
 2   SHAPE_area              327 non-null    float64 
 3   commercial_area         327 non-null    float64 
 4   residential_area        327 non-null    float64 
 5   leisurepark_area        327 non-null    float64 
 6   retail_area             327 non-null    float64 
 7   school_point            327 non-null    float64 
 8   office_point            327 non-null    float64 
 9   bus_point               327 non-null    float64 
 10  railway_point           327 non-null    float64 
 11  subway_point            327 non-null    float64 
 12  tourism_point           327 non-null    float64 
 13  endpointsALL            327 non-null    float64 
 14  PnR_spaces        

In [8]:
trips = trips.drop(columns=['endpoints0', 'endpoints1','endpointsALL'])
trips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   LSOA11CD                327 non-null    object  
 1   USUALRES                327 non-null    float64 
 2   SHAPE_area              327 non-null    float64 
 3   commercial_area         327 non-null    float64 
 4   residential_area        327 non-null    float64 
 5   leisurepark_area        327 non-null    float64 
 6   retail_area             327 non-null    float64 
 7   school_point            327 non-null    float64 
 8   office_point            327 non-null    float64 
 9   bus_point               327 non-null    float64 
 10  railway_point           327 non-null    float64 
 11  subway_point            327 non-null    float64 
 12  tourism_point           327 non-null    float64 
 13  PnR_spaces              327 non-null    float64 
 14  endpoints0_count  

In [9]:
#trips.to_csv('trips_regression.csv', index=False)
trips.to_file("trips_regression.geojson", driver='GeoJSON')